# Van Deemter in Practice: An Interactive Exploration of Chromatographic Efficiency, Flow Rate, and Particle Size

Christian P. Haas, September 2025

## Overview

This notebook turns the van Deemter equation into a hands-on playground. You’ll build intuition for how eddy diffusion (*A*), longitudinal diffusion (*B/u*), and mass transfer (*C·u*) shape plate height *H* as a function of flow rate *u*—and translate that into practical method-design choices for LC (and GC, where noted).

## Learning Objectives

By the end, you will be able to:

* Interpret the *A*, *B/u*, and *C·u* terms and explain how each dominates in different *u*-regimes.
* Identify the optimal flow rate *u*<sub>opt</sub> and corresponding *H*<sub>min</sub>, and relate *H* to plate number (*N = L/H*) and resolution (*R*<sub>s</sub>).
* Compare particle **size** effects on *A* and *C*, and reason about trade-offs among speed, resolution, and pressure limits.
* Make design decisions (e.g., choose *u*, *L*, and particle size) under realistic constraints such as pressure and extra-column broadening.
* (Optional) Use reduced plots (*h–ν*) to compare columns and operating conditions on a common scale.

## Audience & Prerequisites

* **Who:** Students and practitioners in analytical chemistry, chromatography, or method development.
* **Assumed knowledge:** Basic LC/GC terminology (*H, N, u, k′, α, R*<sub>s</sub>*).*
* **No coding required** to use the interactive widgets; minimal Python familiarity helps only if you modify the notebook.

## How to Run

* **In the browser:** Launch via Binder or Colab (use the buttons in the repository README).
* **Locally:** Use JupyterLab with the provided `requirements.txt` or `environment.yml`; then open the notebook and run all cells.

**Estimated time:** 25–40 minutes (core sections); +20 minutes for the optional advanced materials.
**Scope:** LC throughout, with GC notes where diffusion/viscosity dominate differently.
**License:** Content CC BY 4.0; example code MIT.
**Contributions:** Issues and pull requests welcome (see **Contributing** in the repo).


## Theory — a quick introduction

Chromatographic peaks broaden because molecules spread while traveling through the column. The **van Deemter equation** treats this as a simple “budget” of broadening mechanisms and predicts how the **plate height** $H$ (band variance per unit length) depends on **flow rate / linear velocity** $u$:

$$
H(u) \;=\; A \;+\; \frac{B}{u} \;+\; C\,u
$$

* **$A$** — *Eddy diffusion*: different path lengths through a packed bed (≈ constant with $u$). Negligible in open-tubular GC (no packing).
* **$B/u$** — *Longitudinal diffusion*: molecules diffuse along the column axis; strongest at **low $u$**, scales with the molecular diffusion coefficient.
* **$C\,u$** — *Mass-transfer resistance*: analyte lags behind equilibrium between phases; dominates at **high $u$**. Often viewed as lumped contributions from mobile and stationary phases.

The curve $H(u)$ is **U-shaped**. There is a practical sweet spot where total broadening is minimal:

$$
u_{\text{opt}} \;=\; \sqrt{\tfrac{B}{C}}, \qquad H_{\min} \;=\; A \;+\; 2\sqrt{BC}.
$$

Why this matters:

* **Efficiency**: $N = L/H$ (more plates ↔ sharper peaks) for a column of length $L$.
* **Resolution**: better efficiency (smaller $H$) generally improves $R_s$ at fixed selectivity and retention.
* **Design levers**: **particle size** and packing quality influence $A$ and $C$; **diffusion coefficients**, temperature, and solvent affect $B$. Hardware limits (pressure, extra-column effects) can shift the practical operating point away from $u_{\text{opt}}$.

Scope notes:

* We use the classical, **lumped-parameter** form for clarity. Advanced models split $C$ into mobile/stationary terms and use **reduced variables** $h=H/d_p$, $\nu = u\,d_p/D_m$ to compare columns; those appear later where helpful.
* Extra-column broadening isn’t part of van Deemter but is critical in real instruments and will be treated separately.


### Van Deemter — interactive visualization

Use the sliders to explore how **A**, **B/u**, and **C·u** shape the plate-height curve \(H(u)=A+\frac{B}{u}+C\,u\).

- The **shaded background** indicates where each term **dominates** (largest contributor to \(H\) at that \(u\)).  
- Dashed lines show the three terms; the solid line is the **total** \(H(u)\).  
- The vertical dotted line marks \(u_{\mathrm{opt}}=\sqrt{B/C}\); the marker shows \(H_{\min}=A+2\sqrt{BC}\).

*Units are arbitrary for pedagogy; typical LC/GC trends still hold.*


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Checkbox

def van_deemter_plot(A=2.0, B=1.0, C=0.5, u_min=0.05, u_max=3.0, show_terms=True):
    """
    Plot H(u) = A + B/u + C*u and indicate dominance regions for A, B/u, and C*u.
    Parameters are in arbitrary units; u is linear velocity.
    """
    # guard rails
    u_min = max(1e-3, float(u_min))
    u_max = max(u_min + 1e-3, float(u_max))
    A = float(A)
    B = float(B)
    C = float(C)

    # domain
    n = 900
    u = np.linspace(u_min, u_max, n)

    # components and total
    A_arr = A * np.ones_like(u)
    # avoid division warnings by ensuring u_min > 0 (handled above)
    B_arr = B / u
    C_arr = C * u
    H = A_arr + B_arr + C_arr

    # optimum (only meaningful for B>=0 and C>0)
    u_opt = np.sqrt(B / C) if (B > 0 and C > 0) else np.nan
    H_min = A + 2.0 * np.sqrt(B * C) if (B >= 0 and C >= 0) else np.nan

    # figure
    fig, ax = plt.subplots(figsize=(7.2, 4.6))

    # total curve
    ax.plot(u, H, linewidth=2.2, label='H(u) = A + B/u + C·u')

    # optional component curves (dashed)
    if show_terms:
        ax.plot(u, A_arr, linestyle='--', linewidth=1.4, label='A (eddy diffusion)')
        ax.plot(u, B_arr, linestyle='--', linewidth=1.4, label='B/u (longitudinal diffusion)')
        ax.plot(u, C_arr, linestyle='--', linewidth=1.4, label='C·u (mass transfer)')

    # dominance shading: find where each component is the largest contributor
    comps = np.vstack([A_arr, B_arr, C_arr])           # shape: (3, n)
    dom = np.argmax(comps, axis=0)                     # 0:A, 1:B/u, 2:C·u

    # segment the domain where dominance label stays constant
    idx = np.flatnonzero(np.r_[True, np.diff(dom) != 0, True])
    colors = ['C0', 'C1', 'C2']                        # match matplotlib default cycle for A, B/u, C·u
    labels = ['A-dominated', 'B/u-dominated', 'C·u-dominated']
    seen = set()
    for i0, i1 in zip(idx[:-1], idx[1:]):
        d = dom[i0]
        # light shading matching the corresponding line color
        label = labels[d] if d not in seen else None
        ax.axvspan(u[i0], u[i1-1], color=colors[d], alpha=0.08, label=label)
        seen.add(d)

    # mark u_opt and H_min if inside range
    if np.isfinite(u_opt) and (u_min <= u_opt <= u_max) and np.isfinite(H_min):
        ax.axvline(u_opt, linestyle=':', linewidth=1.6)
        ax.plot([u_opt], [H_min], marker='o', markersize=5)
        ax.annotate(r'$u_{\mathrm{opt}}$', xy=(u_opt, H_min),
                    xytext=(6, 8), textcoords='offset points')

    # axes & legend
    ax.set_xlabel('Linear velocity, u (arb. units)')
    ax.set_ylabel('Plate height, H (arb. units)')
    ax.set_title('van Deemter curve with dominance regions')
    ax.grid(True, alpha=0.3)

    # compact legend
    ax.legend(loc='center left', bbox_to_anchor=(1.02, 0.5), frameon=False)
    plt.show()

# Interactive controls
interact(
    van_deemter_plot,
    A=FloatSlider(description='A', value=3.0, min=0.0, max=5.0, step=0.05, readout_format='.2f'),
    B=FloatSlider(description='B', value=1.0, min=0.0, max=5.0, step=0.05, readout_format='.2f'),
    C=FloatSlider(description='C', value=3.0, min=0.0, max=5.0, step=0.01, readout_format='.2f'),
    u_min=FloatSlider(description='u_min', value=0.05, min=0.01, max=1.0, step=0.01, readout_format='.2f'),
    u_max=FloatSlider(description='u_max', value=3.0, min=1.5, max=5.0, step=0.1, readout_format='.1f'),
    show_terms=Checkbox(description='Show A/B/u/C terms', value=True),
)


interactive(children=(FloatSlider(value=3.0, description='A', max=5.0, step=0.05), FloatSlider(value=1.0, desc…

<function __main__.van_deemter_plot(A=2.0, B=1.0, C=0.5, u_min=0.05, u_max=3.0, show_terms=True)>